In [1]:

!pip3 install transformers==4.9.2
!pip3 install datasets==1.5.0
!pip3 install ml-collections==0.1.0
!pip install sentencepiece==0.1.95
!pip install sacrebleu==1.5.1
!pip install unidecode==1.2.0
!pip install rouge_score==0.0.4

In [2]:
!nvidia-smi

Tue Aug 10 21:13:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from pathlib import Path
%cd /
%cd content
!rm -rf stackoverflow-encourages-cheating 
!git clone https://github.com/gabeorlanski/stackoverflow-encourages-cheating.git
%cd stackoverflow-encourages-cheating

/
/content
Cloning into 'stackoverflow-encourages-cheating'...
remote: Enumerating objects: 184, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 184 (delta 75), reused 129 (delta 29), pack-reused 0
Receiving objects: 100% (184/184), 3.63 MiB | 13.01 MiB/s, done.
Resolving deltas: 100% (75/75), done.
/content/stackoverflow-encourages-cheating


# Configuration

Select what configuration you want to use and see the results.

### From the Paper:

* Body => `use_body==True` 
* With Mined => `use_mined==True`
* -NL => `ablation_config=="Only Code"`
* -Code => `ablation_config=="Only NL"`
* -Blocks => `ablation_config=="No Code Blocks"`
* -Inline => `ablation_config=="No Inline"`

In [4]:
use_mined = True #@param {type:"boolean"}
use_body = True #@param {type:"boolean"}
ablation_config = "Full Body" #@param ["Full Body", "Only NL", "Only Code", "No Code Blocks", "No Inline"]
batch_size = 12 #@param {type:"integer"}

preprocessor_kwargs = {}
if use_body:
    if ablation_config == "Only NL":
        preprocessor_kwargs['remove_all_code']=True
    elif ablation_config == "Only Code":
        preprocessor_kwargs["use_only_code"]=True
    elif ablation_config == "No Code Blocks":
        preprocessor_kwargs["remove_code_blocks"]=True
    elif ablation_config == "No Inline":
        preprocessor_kwargs["remove_inline_code"]=True

In [5]:
from transformers import (
    AutoTokenizer, PreTrainedTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer,
    Seq2SeqTrainingArguments, TrainingArguments, Trainer, AutoConfig,
    AutoModelForSequenceClassification
)
from datasets import load_dataset, Metric, load_metric, list_metrics
import torch
import ml_collections
import transformers
from numpy.random import default_rng
import numpy as np
import json
import re
import random

seed = 1995
rng = default_rng(seed)
def setSeed():
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
setSeed()

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_name = "facebook/bart-base"

In [7]:
from src import processor, evaluation
from src.evaluation import utility
from src.processor import *
from src.common.training_util import loadDatasets, processRawDatasets
from src.evaluation import *
from src.processor.common import *

max_length= 128 if not use_body else 512

# Dataset Preprocessing

Load the `CodeGenerationProcessor` with the current config. 

**NOTE:** Google Colab is Python 3.*7* whereas the `py3_asdl.grammar` is for python 3.*8*.
 Thankfully it is not used here but do be aware of that.

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
preprocessor = CodeGenerationProcessor(
    'best',
    f'bartBase{"wMined" if use_mined else ""}',
    tokenizer,
    Path('data','html_tags.txt'),
    Path('data','py3_asdl.grammar'),
    max_len=max_length,
    target_max_len=128,
    use_body=use_body,
    **preprocessor_kwargs
)
print(preprocessor.name)

best_bartBasewMined_512in_128out_Intent_Snippet_body


Load the datasets and process them

In [9]:

raw_datasets = loadDatasets(
        preprocessor=preprocessor,
        use_filter=False,
        load_dataset_args={'skip_api': True},
)

datasets, new_splits = processRawDatasets(
    raw_datasets,
    preprocessor,
    num_procs=4,
    shuffle_seed=21,
    val_size=.1,
    combine_mined_train=use_mined
)

for split_name, split in new_splits.items():
    raw_datasets[split_name] = split

Using custom data configuration default-47d77f710bd55aad
Reusing dataset base_dataset (/root/.cache/huggingface/datasets/base_dataset/default-47d77f710bd55aad/1.1.0/096d241636bbd19e0f9e8391fc6cdc57a7d1e5af9d67cbbf5eb39dbcd8a5a5a3)


Processing split named 'test'


Loading cached processed dataset at /root/.cache/huggingface/datasets/base_dataset/default-47d77f710bd55aad/1.1.0/096d241636bbd19e0f9e8391fc6cdc57a7d1e5af9d67cbbf5eb39dbcd8a5a5a3/cache-5e78011def695fc8.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/base_dataset/default-47d77f710bd55aad/1.1.0/096d241636bbd19e0f9e8391fc6cdc57a7d1e5af9d67cbbf5eb39dbcd8a5a5a3/cache-9be6a85155cdadfa.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/base_dataset/default-47d77f710bd55aad/1.1.0/096d241636bbd19e0f9e8391fc6cdc57a7d1e5af9d67cbbf5eb39dbcd8a5a5a3/cache-e096895538659b27.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/base_dataset/default-47d77f710bd55aad/1.1.0/096d241636bbd19e0f9e8391fc6cdc57a7d1e5af9d67cbbf5eb39dbcd8a5a5a3/cache-86d3d4bb7c8cee1d.arrow
Loading cached split indices for dataset at /root/.cache/huggingface/datasets/base_dataset/default-47d77f710bd55aad/1.1.0/096d241636bbd19e0f9e8391fc6cdc57a7d1e5af9d67cb

Processing split named 'val'


Loading cached processed dataset at /root/.cache/huggingface/datasets/base_dataset/default-47d77f710bd55aad/1.1.0/096d241636bbd19e0f9e8391fc6cdc57a7d1e5af9d67cbbf5eb39dbcd8a5a5a3/cache-53a48fc33e87f5e1.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/base_dataset/default-47d77f710bd55aad/1.1.0/096d241636bbd19e0f9e8391fc6cdc57a7d1e5af9d67cbbf5eb39dbcd8a5a5a3/cache-a35e1384332420b3.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/base_dataset/default-47d77f710bd55aad/1.1.0/096d241636bbd19e0f9e8391fc6cdc57a7d1e5af9d67cbbf5eb39dbcd8a5a5a3/cache-e794a90e4af7d71d.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/base_dataset/default-47d77f710bd55aad/1.1.0/096d241636bbd19e0f9e8391fc6cdc57a7d1e5af9d67cbbf5eb39dbcd8a5a5a3/cache-c71490221b5ef5ef.arrow


Processing split named 'train'


Loading cached processed dataset at /root/.cache/huggingface/datasets/base_dataset/default-47d77f710bd55aad/1.1.0/096d241636bbd19e0f9e8391fc6cdc57a7d1e5af9d67cbbf5eb39dbcd8a5a5a3/cache-1379a087592e1b1f.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/base_dataset/default-47d77f710bd55aad/1.1.0/096d241636bbd19e0f9e8391fc6cdc57a7d1e5af9d67cbbf5eb39dbcd8a5a5a3/cache-60efc99d03810b2f.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/base_dataset/default-47d77f710bd55aad/1.1.0/096d241636bbd19e0f9e8391fc6cdc57a7d1e5af9d67cbbf5eb39dbcd8a5a5a3/cache-fa99a0785b322207.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/base_dataset/default-47d77f710bd55aad/1.1.0/096d241636bbd19e0f9e8391fc6cdc57a7d1e5af9d67cbbf5eb39dbcd8a5a5a3/cache-16ba7749312ebe8c.arrow


## Sample Inputs for the Data

In [10]:
num_to_print = 5
print(f"Data from {preprocessor.name}:")
print()
for k in ['train','val','test']:
    print(f"{k} ({datasets[k].num_rows} examples):")
    print(f"\tFirst {num_to_print} ids in {k}: {datasets[k]['question_id'][:num_to_print]}")
    print(f"\tFirst {num_to_print} text:")
    for i in range(num_to_print): 
        decoded = repr(tokenizer.decode(datasets[k][i]['input_ids']))
        print(f"\t\t{decoded[:128]}")


Data from best_bartBasewMined_512in_128out_Intent_Snippet_body:

train (11127 examples):
	First 5 ids in train: ['4576115', '280222', '4967580', '32800623', '13114512']
	First 5 text:
		"<s>convert a list to a dictionary in python  Let's say I have a list  a   in Python whose entries conveniently map to a diction
		"<s>python - sort a list of nested lists  I have input consisting of a list of nested lists like this:\n l = [[[[[39]]]], [1, 2,
		'<s>how to get the size of a string in python?  For example, I get a string:\n str = "please answer my question"\n\n I want to w
		"<s>how to get the fft of a numpy array to work?  I'm reading a specific column of a csv file as a numpy array. When I try to do
		"<s>calculating difference between two rows in python / pandas  In python, how can I reference previous row and calculate someth
val (1237 examples):
	First 5 ids in val: ['28134319', '36368252', '3258573', '30108372', '6086047']
	First 5 text:
		"<s>remove first and last lines of string `

# Setting up the Model + Evaluator

In [11]:

config = AutoConfig.from_pretrained(model_name)
config.max_length = preprocessor.max_target_len

num_return = 4
gen_kwargs ={
    'early_stopping':True,
    'num_beams': num_return,
    'length_penalty': .9,
    'num_return_sequences':num_return,
}
for k,v in gen_kwargs.items():
  setattr(config,k,v)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    config=config
)

model.resize_token_embeddings(preprocessor.vocab_size).to(device)
if not hasattr(model, 'keys_to_ignore_at_inference'):
    model.keys_to_ignore_at_inference = []
model.keys_to_ignore_at_inference.extend(preprocessor.ignore_keys)



In [12]:

evaluator = Seq2Seq.CodeGenerationEvaluator(tokenizer, device,smooth_bleu=True)

In [17]:
def trainingArgs(logging_dir, label_names = None):
    if label_names is None:
        label_names = ['labels']
    _config = ml_collections.ConfigDict()
    _config.output_dir = logging_dir
    _config.evaluation_strategy = "epoch"
    _config.save_strategy = "epoch"
    _config.per_device_train_batch_size = batch_size
    _config.per_device_eval_batch_size = batch_size
    _config.logging_dir = logging_dir
    _config.save_total_limit = 2
    _config.learning_rate = 5e-5
    _config.seed = seed
    _config.num_train_epochs = 10
    _config.dataloader_num_workers = 0
    _config.label_names = label_names
    _config.no_cuda = False
    _config.load_best_model_at_end = True
    _config.group_by_length =True
    _config.warmup_ratio = 0.05
    _config.logging_strategy='epoch'
    return _config

# Training

In [18]:
args_dict = trainingArgs("./experiments/")
training_args = Seq2SeqTrainingArguments(**args_dict)
training_args.predict_with_generate= True
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer,
    pad_to_multiple_of=4
)
evaluator.minimal=True
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    compute_metrics=evaluator,
    train_dataset=datasets['train'],
    eval_dataset=datasets['val'],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()
trainer.save_state()

The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: question_id, idx, slot_map.
***** Running training *****
  Num examples = 11127
  Num Epochs = 10
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 12
  Gradient Accumulation steps = 1
  Total optimization steps = 9280


Epoch,Training Loss,Validation Loss,Bleu,Bleu-unigram-precision,Bleu-bigram-precision
1,2.635700,1.843910,3.137188,32.732628,8.294759


The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: question_id, idx, slot_map.
***** Running Evaluation *****
  Num examples = 1237
  Batch size = 12
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
Saving model checkpoint to ./experiments/checkpoint-928
Configuration saved in ./experiments/checkpoint-928/config.json
Model weights saved in ./experiments/checkpoint-928/pytorch_model.bin
tokenizer config fil

# Evaluation

In [ ]:
num_return = 4
gen_kwargs ={
    'early_stopping':True,
    'num_beams': num_return, 
    'length_penalty': .9,
    'num_return_sequences':num_return,
}

In [ ]:
evaluator.minimal=False
utility.evaluateExperiments(
    trainer, 
    preprocessor, 
    datasets,
    raw_datasets,
    evaluator, 
    Path(),
    Path('experiments'),
    allow_overwrite=True,
    batch_size=16,
    gen_kwargs=gen_kwargs,
)
